In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration for 6-month horizon
########################
ticker = 'COST'
start_date = '2020-01-01'
end_date = '2024-01-01'

n_steps = 5
window_size = 200
step_size = 10
initial_epochs = 100
update_epochs = 100
initial_lr = 0.0005
update_lr = 0.0002
train_ratio = 0.9
prediction_horizon = 126  # 6-month horizon

if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")

def load_full_data_helper(ticker, start_date, end_date, window_size, prediction_horizon):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    # Shift by prediction_horizon days
    stock_data['Close_future'] = stock_data['Close'].shift(-prediction_horizon)
    stock_data.dropna(subset=['Close_future'], inplace=True)

    # Compute features: BB_Upper, BB_Lower, Volatility_30
    # Using a 90-day Bollinger Band as user snippet suggests
    upper_band, middle_band, lower_band = talib.BBANDS(stock_data['Close'], timeperiod=90, nbdevup=2, nbdevdn=2)
    stock_data['BB_Upper'] = upper_band
    stock_data['BB_Lower'] = lower_band

    stock_data['Returns'] = stock_data['Close'].pct_change()
    stock_data['Volatility_30'] = stock_data['Returns'].rolling(90).std()

    # Drop rows with NaNs created by indicators
    stock_data.dropna(subset=['BB_Upper', 'BB_Lower', 'Volatility_30', 'Close_future'], inplace=True)

    # Check if we still have enough data
    if len(stock_data) < window_size:
        return None, None, None

    # Features are now BB_Upper, BB_Lower, Volatility_30
    X_feat = stock_data[['BB_Upper', 'BB_Lower', 'Volatility_30']].values
    y_raw = stock_data['Close_future'].values
    dates = stock_data['Date'].values

    return X_feat, y_raw, dates

def load_full_data(ticker, start_date, end_date, window_size, prediction_horizon):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size, prediction_horizon)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)
    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)

def create_lstm_model(n_units, learning_rate, n_steps, n_features, n_layers=1):
    inputs = Input(shape=(n_steps, n_features))
    x = LSTM(n_units, activation='relu', return_sequences=(n_layers > 1))(inputs)
    if n_layers > 1:
        x = LSTM(n_units, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    return model  # Note: we compile after loading weights

def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None,
                    initial_epochs=100, update_epochs=100,
                    initial_lr=0.0005, update_lr=0.0002,
                    n_units=16, batch_size=4):
    total_samples = len(X_samples)
    train_ratio = 0.9
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if X_train.size == 0 or X_val.size == 0:
        return None, np.nan

    n_steps = X_train.shape[1]
    n_features = X_train.shape[2]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    model = create_lstm_model(n_units, lr, n_steps, n_features, n_layers=1)

    # Load weights before compile to avoid optimizer warnings
    if prev_weights is not None:
        model.load_weights(prev_weights)

    # Compile after loading weights
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape

def rolling_training_early_stopping(ticker, start_date, end_date,
                                    n_steps=5, window_size=200, step_size=10,
                                    initial_epochs=100, update_epochs=100,
                                    initial_lr=0.0005, update_lr=0.0002,
                                    n_units=16, batch_size=4,
                                    prediction_horizon=126):

    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size, prediction_horizon)
    if X_full is None:
        return None, None

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        return None, None
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                                      initial_training=True, prev_weights=None,
                                      initial_epochs=initial_epochs, update_epochs=update_epochs,
                                      initial_lr=initial_lr, update_lr=update_lr,
                                      n_units=n_units, batch_size=batch_size)

    if model is None:
        return None, None

    weights_path = f"rolling_error_analysis_results/{ticker}_initial_6months.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                                          initial_training=False, prev_weights=weights_path,
                                          initial_epochs=initial_epochs, update_epochs=update_epochs,
                                          initial_lr=initial_lr, update_lr=update_lr,
                                          n_units=n_units, batch_size=batch_size)

        if model is None:
            break

        weights_path = f"rolling_error_analysis_results/{ticker}_iteration_{iteration}_6months.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])
        iteration += 1

    if len(predictions) == 0:
        return None, None

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0
    overall_mape = df_results['MAPE'].mean()

    return overall_mape, df_results

if __name__ == "__main__":
    mape, results_df = rolling_training_early_stopping(
        ticker,
        start_date=start_date,
        end_date=end_date,
        n_steps=n_steps,
        window_size=window_size,
        step_size=step_size,
        initial_epochs=initial_epochs,
        update_epochs=update_epochs,
        initial_lr=initial_lr,
        update_lr=update_lr,
        n_units=16,
        batch_size=4,
        prediction_horizon=126
    )

    if mape is not None:
        print(f"[INFO] 6-month horizon rolling training for {ticker} completed. Overall MAPE: {mape:.2f}%")
        results_df.to_csv("rolling_error_analysis_results/cost_6months_results1.csv", index=False)
        print("[INFO] Detailed results saved to cost_6months_results1.csv")
    else:
        print("[WARN] Not enough data or no predictions made for COST with 6-month horizon.")


[*********************100%%**********************]  1 of 1 completed
[INFO] 6-month horizon rolling training for COST completed. Overall MAPE: 21.59%
[INFO] Detailed results saved to cost_6months_results1.csv


In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration for 6-month horizon
########################
ticker = 'COST'
start_date = '2020-01-01'
end_date = '2024-01-01'

n_steps = 1
window_size = 10
step_size = 5
initial_epochs = 100
update_epochs = 100
initial_lr = 0.0005
update_lr = 0.0002
train_ratio = 0.9
prediction_horizon = 126  # 6-month horizon

if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")

def load_full_data_helper(ticker, start_date, end_date, window_size, prediction_horizon):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    # Shift by prediction_horizon days
    stock_data['Close_future'] = stock_data['Close'].shift(-prediction_horizon)
    stock_data.dropna(subset=['Close_future'], inplace=True)

    # Compute features: BB_Upper, BB_Lower, Volatility_30
    # Using a 90-day Bollinger Band as user snippet suggests
    upper_band, middle_band, lower_band = talib.BBANDS(stock_data['Close'], timeperiod=90, nbdevup=2, nbdevdn=2)
    stock_data['BB_Upper'] = upper_band
    stock_data['BB_Lower'] = lower_band

    stock_data['Returns'] = stock_data['Close'].pct_change()
    stock_data['Volatility_30'] = stock_data['Returns'].rolling(90).std()

    # Drop rows with NaNs created by indicators
    stock_data.dropna(subset=['BB_Upper', 'BB_Lower', 'Volatility_30', 'Close_future'], inplace=True)

    # Check if we still have enough data
    if len(stock_data) < window_size:
        return None, None, None

    # Features are now BB_Upper, BB_Lower, Volatility_30
    X_feat = stock_data[['BB_Upper', 'BB_Lower', 'Volatility_30']].values
    y_raw = stock_data['Close_future'].values
    dates = stock_data['Date'].values

    return X_feat, y_raw, dates

def load_full_data(ticker, start_date, end_date, window_size, prediction_horizon):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size, prediction_horizon)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)
    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)

def create_lstm_model(n_units, learning_rate, n_steps, n_features, n_layers=1):
    inputs = Input(shape=(n_steps, n_features))
    x = LSTM(n_units, activation='relu', return_sequences=(n_layers > 1))(inputs)
    if n_layers > 1:
        x = LSTM(n_units, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    return model  # Note: we compile after loading weights

def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None,
                    initial_epochs=100, update_epochs=100,
                    initial_lr=0.0005, update_lr=0.0002,
                    n_units=16, batch_size=4):
    total_samples = len(X_samples)
    train_ratio = 0.9
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if X_train.size == 0 or X_val.size == 0:
        return None, np.nan

    n_steps = X_train.shape[1]
    n_features = X_train.shape[2]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    model = create_lstm_model(n_units, lr, n_steps, n_features, n_layers=1)

    # Load weights before compile to avoid optimizer warnings
    if prev_weights is not None:
        model.load_weights(prev_weights)

    # Compile after loading weights
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape

def rolling_training_early_stopping(ticker, start_date, end_date,
                                    n_steps=5, window_size=200, step_size=10,
                                    initial_epochs=100, update_epochs=100,
                                    initial_lr=0.0005, update_lr=0.0002,
                                    n_units=16, batch_size=4,
                                    prediction_horizon=126):

    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size, prediction_horizon)
    if X_full is None:
        return None, None

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        return None, None
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                                      initial_training=True, prev_weights=None,
                                      initial_epochs=initial_epochs, update_epochs=update_epochs,
                                      initial_lr=initial_lr, update_lr=update_lr,
                                      n_units=n_units, batch_size=batch_size)

    if model is None:
        return None, None

    weights_path = f"rolling_error_analysis_results/{ticker}_initial_6months.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                                          initial_training=False, prev_weights=weights_path,
                                          initial_epochs=initial_epochs, update_epochs=update_epochs,
                                          initial_lr=initial_lr, update_lr=update_lr,
                                          n_units=n_units, batch_size=batch_size)

        if model is None:
            break

        weights_path = f"rolling_error_analysis_results/{ticker}_iteration_{iteration}_6months.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])
        iteration += 1

    if len(predictions) == 0:
        return None, None

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0
    overall_mape = df_results['MAPE'].mean()

    return overall_mape, df_results

if __name__ == "__main__":
    mape, results_df = rolling_training_early_stopping(
        ticker,
        start_date=start_date,
        end_date=end_date,
        n_steps=n_steps,
        window_size=window_size,
        step_size=step_size,
        initial_epochs=initial_epochs,
        update_epochs=update_epochs,
        initial_lr=initial_lr,
        update_lr=update_lr,
        n_units=16,
        batch_size=4,
        prediction_horizon=126
    )

    if mape is not None:
        print(f"[INFO] 6-month horizon rolling training for {ticker} completed. Overall MAPE: {mape:.2f}%")
        results_df.to_csv("rolling_error_analysis_results/cost_6months_results1.csv", index=False)
        print("[INFO] Detailed results saved to cost_6months_results1.csv")
    else:
        print("[WARN] Not enough data or no predictions made for COST with 6-month horizon.")


[*********************100%%**********************]  1 of 1 completed
[INFO] 6-month horizon rolling training for COST completed. Overall MAPE: 2.10%
[INFO] Detailed results saved to cost_6months_results1.csv


In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
import os
import math

########################
# Configuration for 6-month horizon
########################
ticker = 'COST'
start_date = '2020-01-01'
end_date = '2024-01-01'

window_size = 10
step_size = 5
prediction_horizon = 126  # Predicting 6 months (126 trading days) ahead
history_length = 90  # 90 days of historical data

if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")

def load_full_data(ticker, start_date, end_date, history_length, prediction_horizon):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    stock_data['Close_future'] = stock_data['Close'].shift(-prediction_horizon)
    stock_data.dropna(subset=['Close_future'], inplace=True)

    # Use the last 90 days of close prices as features
    X = []
    y = stock_data['Close_future'].values
    dates = stock_data['Date'].values

    for i in range(history_length, len(stock_data)):
        X.append(stock_data['Close'].iloc[i-history_length:i].values)

    X = np.array(X)
    y = y[history_length:]
    dates = dates[history_length:]

    return X, y, dates

def rolling_predictions(ticker, start_date, end_date, history_length, window_size, step_size, prediction_horizon):
    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, history_length, prediction_horizon)
    if X_full is None:
        print(f"[WARN] Not enough data for {ticker}.")
        return None

    start_idx = 0
    results = []

    while start_idx + window_size <= len(X_full):
        X_window = X_full[start_idx:start_idx + window_size]
        y_window = y_full[start_idx:start_idx + window_size]
        dates_window = dates_full[start_idx:start_idx + window_size]

        for i in range(len(X_window)):
            X_sample = X_window[i]
            y_actual = y_window[i]
            date = dates_window[i]

            # SMA Prediction
            sma_pred = np.mean(X_sample[-2:])

            # EMA Prediction
            alpha = 2 / (2 + 1)
            ema_pred = alpha * X_sample[-1] + (1 - alpha) * X_sample[-2]

            # Linear Regression Prediction
            X_train = np.array([[-2], [-1]])  # Days n-2 and n-1
            y_train = X_sample[-2:]
            lr_model = LinearRegression()
            lr_model.fit(X_train, y_train)
            lr_pred = lr_model.predict([[0]])[0]  # Predict day n

            results.append({
                'Date': date,
                'Actual': y_actual,
                'SMA_Predicted': sma_pred,
                'EMA_Predicted': ema_pred,
                'LR_Predicted': lr_pred
            })

        start_idx += step_size

    df_results = pd.DataFrame(results)
    df_results['SMA_Absolute_Error'] = np.abs(df_results['Actual'] - df_results['SMA_Predicted'])
    df_results['EMA_Absolute_Error'] = np.abs(df_results['Actual'] - df_results['EMA_Predicted'])
    df_results['LR_Absolute_Error'] = np.abs(df_results['Actual'] - df_results['LR_Predicted'])
    df_results['SMA_MAPE'] = (df_results['SMA_Absolute_Error'] / np.abs(df_results['Actual'])) * 100
    df_results['EMA_MAPE'] = (df_results['EMA_Absolute_Error'] / np.abs(df_results['Actual'])) * 100
    df_results['LR_MAPE'] = (df_results['LR_Absolute_Error'] / np.abs(df_results['Actual'])) * 100

    overall_sma_mape = df_results['SMA_MAPE'].mean()
    overall_ema_mape = df_results['EMA_MAPE'].mean()
    overall_lr_mape = df_results['LR_MAPE'].mean()

    print(f"[INFO] {ticker}: SMA MAPE: {overall_sma_mape:.2f}%, EMA MAPE: {overall_ema_mape:.2f}%, LR MAPE: {overall_lr_mape:.2f}%")

    return df_results

if __name__ == "__main__":
    results_df = rolling_predictions(ticker, start_date, end_date, history_length, window_size, step_size, prediction_horizon)

    if results_df is not None:
        results_path = "rolling_error_analysis_results/costco_results.csv"
        results_df.to_csv(results_path, index=False)
        print(f"[INFO] Detailed results saved to {results_path}")
    else:
        print("[WARN] No predictions were made.")

[*********************100%%**********************]  1 of 1 completed
[INFO] COST: SMA MAPE: 12.39%, EMA MAPE: 12.39%, LR MAPE: 12.47%
[INFO] Detailed results saved to rolling_error_analysis_results/costco_results.csv


In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.metrics import mean_absolute_percentage_error
import os
import math
import time

########################
# Configuration for 6-month horizon
########################
ticker = 'COST'
start_date = '2020-01-01'
end_date = '2024-01-01'

n_steps = 5
window_size = 200
step_size = 10
prediction_horizon = 126  # approx 6 months

if not os.path.exists("baseline_sma_6months_results"):
    os.makedirs("baseline_sma_6months_results")

def load_data(ticker, start_date, end_date, window_size, prediction_horizon):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    # Shift by prediction_horizon days
    stock_data['Close_future'] = stock_data['Close'].shift(-prediction_horizon)
    stock_data.dropna(subset=['Close_future'], inplace=True)

    X = stock_data['Close'].values
    y = stock_data['Close_future'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None

    return X, y, dates

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def create_samples(dataX, dataY, n_steps):
        X, y, dates = [], [], []
        for i in range(len(dataX)-n_steps):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
            dates.append(dates_window[i+n_steps-1])
        return np.array(X), np.array(y), np.array(dates)

    X_samples, y_samples, sample_dates = create_samples(X_window, y_window, n_steps)
    if X_samples.size == 0:
        return None, None, None, None

    return X_samples, y_samples, sample_dates, (X_window, y_window, dates_window)

def baseline_sma_prediction(X_samples, n_steps):
    predictions = []
    for i in range(len(X_samples)):
        pred = np.mean(X_samples[i])
        predictions.append(pred)
    return np.array(predictions)

def rolling_baseline_sma_6months(ticker, window_size, step_size, n_steps, prediction_horizon):
    X_full, y_full, dates_full = load_data(ticker, start_date, end_date, window_size, prediction_horizon)
    if X_full is None:
        print(f"[WARN] Not enough data for {ticker}.")
        return None

    start_time = time.time()
    start_idx = 0

    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print(f"[WARN] Initial window not valid for {ticker}.")
        return None
    X_samples, y_samples, sample_dates, _ = window_data

    predictions = baseline_sma_prediction(X_samples, n_steps)
    all_predictions = list(predictions)
    all_actuals = list(y_samples)
    all_dates = list(sample_dates)

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print(f"[INFO] No more windows can be extracted for {ticker}.")
            break
        X_samples, y_samples, sample_dates, _ = window_data
        predictions = baseline_sma_prediction(X_samples, n_steps)
        all_predictions.extend(predictions)
        all_actuals.extend(y_samples)
        all_dates.extend(sample_dates)
        iteration += 1

    all_predictions = np.array(all_predictions)
    all_actuals = np.array(all_actuals)
    mape = mean_absolute_percentage_error(all_actuals, all_predictions)*100.0

    df_results = pd.DataFrame({
        'Date': all_dates,
        'Actual': all_actuals,
        'Predicted': all_predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv(f"baseline_sma_6months_results/{ticker}_6months_sma_baseline_results.csv", index=False)
    print(f"[INFO] 6-month horizon SMA baseline for {ticker}: Overall MAPE: {mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] {ticker}: Total elapsed time: {elapsed:.2f}s")

    return mape

if __name__ == "__main__":
    mape = rolling_baseline_sma_6months(ticker, window_size, step_size, n_steps, prediction_horizon)
    if mape is not None:
        print(f"[INFO] 6-month horizon SMA baseline completed for {ticker}. MAPE={mape:.2f}%")
    else:
        print("[WARN] Could not complete SMA baseline for 6-month horizon.")


[*********************100%%**********************]  1 of 1 completed
[INFO] No more windows can be extracted for COST.
[INFO] 6-month horizon SMA baseline for COST: Overall MAPE: 12.33%
[INFO] COST: Total elapsed time: 0.10s
[INFO] 6-month horizon SMA baseline completed for COST. MAPE=12.33%


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration for 6-month horizon
########################

stocks = [
    'COST', 'UL', 'AMGN', 'UNH', 'WAT', 'UPS', 'LPX', 'WM', 'NVDA',
    'GOOGL', 'MSFT', 'AXP', 'BLK', 'BRK-B', 'NEE', 'XOM', 'CNI'
]

start_date = '2020-01-01'
end_date = '2024-01-01'

# Hyperparameters (easily editable)
n_steps = 5
window_size = 200
step_size = 10
initial_epochs = 100
update_epochs = 100
initial_lr = 0.0005
update_lr = 0.0002
train_ratio = 0.9
n_units = 16
batch_size = 4
prediction_horizon = 126  # 6-month horizon

if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")

def load_full_data_helper(ticker, start_date, end_date, window_size, prediction_horizon):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    stock_data['Close_future'] = stock_data['Close'].shift(-prediction_horizon)
    stock_data.dropna(subset=['Close_future'], inplace=True)

    upper_band, middle_band, lower_band = talib.BBANDS(stock_data['Close'], timeperiod=90, nbdevup=2, nbdevdn=2)
    stock_data['BB_Upper'] = upper_band
    stock_data['BB_Lower'] = lower_band
    stock_data['Returns'] = stock_data['Close'].pct_change()
    stock_data['Volatility_30'] = stock_data['Returns'].rolling(90).std()
    stock_data.dropna(subset=['BB_Upper', 'BB_Lower', 'Volatility_30', 'Close_future'], inplace=True)

    if len(stock_data) < window_size:
        return None, None, None

    X_feat = stock_data[['BB_Upper', 'BB_Lower', 'Volatility_30']].values
    y_raw = stock_data['Close_future'].values
    dates = stock_data['Date'].values
    return X_feat, y_raw, dates

def load_full_data(ticker, start_date, end_date, window_size, prediction_horizon):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size, prediction_horizon)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)
    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)

def create_lstm_model(n_units, learning_rate, n_steps, n_features, n_layers=1):
    inputs = Input(shape=(n_steps, n_features))
    x = LSTM(n_units, activation='relu', return_sequences=(n_layers > 1))(inputs)
    if n_layers > 1:
        x = LSTM(n_units, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    return model

def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None,
                    initial_epochs=100, update_epochs=100,
                    initial_lr=0.0005, update_lr=0.0002,
                    n_units=16, batch_size=4, train_ratio=0.9):
    total_samples = len(X_samples)
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if X_train.size == 0 or X_val.size == 0:
        return None, np.nan

    n_steps_local = X_train.shape[1]
    n_features = X_train.shape[2]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    model = create_lstm_model(n_units, lr, n_steps_local, n_features, n_layers=1)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape

def rolling_training_early_stopping(ticker, start_date, end_date,
                                    n_steps=5, window_size=200, step_size=10,
                                    initial_epochs=100, update_epochs=100,
                                    initial_lr=0.0005, update_lr=0.0002,
                                    n_units=16, batch_size=4,
                                    prediction_horizon=126, train_ratio=0.9):
    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size, prediction_horizon)
    if X_full is None:
        return None

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        return None
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                                      initial_training=True, prev_weights=None,
                                      initial_epochs=initial_epochs, update_epochs=update_epochs,
                                      initial_lr=initial_lr, update_lr=update_lr,
                                      n_units=n_units, batch_size=batch_size, train_ratio=train_ratio)

    if model is None:
        return None

    weights_path = f"rolling_error_analysis_results/{ticker}_initial_6months.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                                          initial_training=False, prev_weights=weights_path,
                                          initial_epochs=initial_epochs, update_epochs=update_epochs,
                                          initial_lr=initial_lr, update_lr=update_lr,
                                          n_units=n_units, batch_size=batch_size, train_ratio=train_ratio)

        if model is None:
            break

        weights_path = f"rolling_error_analysis_results/{ticker}_iteration_{iteration}_6months.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])
        iteration += 1

    if len(predictions) == 0:
        return None

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0
    overall_mape = df_results['MAPE'].mean()

    return overall_mape

if __name__ == "__main__":
    # Multi-ticker run: run for each stock and save MAPEs
    results = []
    for s in stocks:
        mape = rolling_training_early_stopping(
            s,
            start_date=start_date,
            end_date=end_date,
            n_steps=n_steps,
            window_size=window_size,
            step_size=step_size,
            initial_epochs=initial_epochs,
            update_epochs=update_epochs,
            initial_lr=initial_lr,
            update_lr=update_lr,
            n_units=n_units,
            batch_size=batch_size,
            prediction_horizon=prediction_horizon,
            train_ratio=train_ratio
        )
        if mape is not None:
            results.append({'Stock': s, 'Overall_MAPE': mape})
        else:
            results.append({'Stock': s, 'Overall_MAPE': np.nan})

    results_df = pd.DataFrame(results)
    results_df.to_csv('stock_mapes_6months.csv', index=False)
    print("[INFO] All stock MAPEs saved to stock_mapes_6months.csv")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.metrics import mean_absolute_percentage_error
import os
import math
import time

########################
# Configuration for 6-month horizon
########################

stocks = [
    'COST', 'UL', 'AMGN', 'UNH', 'WAT', 'UPS', 'LPX', 'WM', 'NVDA',
    'GOOGL', 'MSFT', 'AXP', 'BLK', 'BRK-B', 'NEE', 'XOM', 'CNI'
]

start_date = '2020-01-01'
end_date = '2024-01-01'

n_steps = 5
window_size = 200
step_size = 10
prediction_horizon = 126  # approx 6 months

if not os.path.exists("baseline_sma_6months_results"):
    os.makedirs("baseline_sma_6months_results")

def load_data(ticker, start_date, end_date, window_size, prediction_horizon):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None
    stock_data['Close_future'] = stock_data['Close'].shift(-prediction_horizon)
    stock_data.dropna(subset=['Close_future'], inplace=True)

    X = stock_data['Close'].values
    y = stock_data['Close_future'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None

    return X, y, dates

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def create_samples(dataX, dataY, n_steps):
        X, y, dates = [], [], []
        for i in range(len(dataX)-n_steps):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
            dates.append(dates_window[i+n_steps-1])
        return np.array(X), np.array(y), np.array(dates)

    X_samples, y_samples, sample_dates = create_samples(X_window, y_window, n_steps)
    if X_samples.size == 0:
        return None, None, None, None

    return X_samples, y_samples, sample_dates, (X_window, y_window, dates_window)

def baseline_sma_prediction(X_samples, n_steps):
    predictions = []
    for i in range(len(X_samples)):
        pred = np.mean(X_samples[i])
        predictions.append(pred)
    return np.array(predictions)

def rolling_baseline_sma_6months(ticker, window_size, step_size, n_steps, prediction_horizon):
    X_full, y_full, dates_full = load_data(ticker, start_date, end_date, window_size, prediction_horizon)
    if X_full is None:
        print(f"[WARN] Not enough data for {ticker}.")
        return None

    start_time = time.time()
    start_idx = 0

    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print(f"[WARN] Initial window not valid for {ticker}.")
        return None
    X_samples, y_samples, sample_dates, _ = window_data

    predictions = baseline_sma_prediction(X_samples, n_steps)
    all_predictions = list(predictions)
    all_actuals = list(y_samples)
    all_dates = list(sample_dates)

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print(f"[INFO] No more windows can be extracted for {ticker}.")
            break
        X_samples, y_samples, sample_dates, _ = window_data
        predictions = baseline_sma_prediction(X_samples, n_steps)
        all_predictions.extend(predictions)
        all_actuals.extend(y_samples)
        all_dates.extend(sample_dates)
        iteration += 1

    all_predictions = np.array(all_predictions)
    all_actuals = np.array(all_actuals)
    mape = mean_absolute_percentage_error(all_actuals, all_predictions)*100.0

    df_results = pd.DataFrame({
        'Date': all_dates,
        'Actual': all_actuals,
        'Predicted': all_predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv(f"baseline_sma_6months_results/{ticker}_6months_sma_baseline_results.csv", index=False)
    print(f"[INFO] 6-month horizon SMA baseline for {ticker}: Overall MAPE: {mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] {ticker}: Total elapsed time: {elapsed:.2f}s")

    return mape

if __name__ == "__main__":
    results = []
    for s in stocks:
        mape = rolling_baseline_sma_6months(s, window_size, step_size, n_steps, prediction_horizon)
        if mape is not None:
            results.append({'Stock': s, 'Overall_MAPE': mape})
        else:
            results.append({'Stock': s, 'Overall_MAPE': np.nan})

    results_df = pd.DataFrame(results)
    results_df.to_csv('stock_mapes_6months_sma_baseline.csv', index=False)
    print("[INFO] All SMA baseline MAPEs for 6-month horizon saved to stock_mapes_6months_sma_baseline.csv")


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration for 6-month horizon
########################

ticker = 'COST'
start_date = '2020-01-01'
end_date = '2024-01-01'

# Hyperparameters (easily editable)
n_steps = 5
window_size = 200
step_size = 10
initial_epochs = 100
update_epochs = 100
initial_lr = 0.0005
update_lr = 0.0002
train_ratio = 0.9
n_units = 16
batch_size = 4
prediction_horizon = 126  # 6-month horizon

if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")

def load_full_data_helper(ticker, start_date, end_date, window_size, prediction_horizon):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    stock_data['Close_future'] = stock_data['Close'].shift(-prediction_horizon)
    stock_data.dropna(subset=['Close_future'], inplace=True)
    upper_band, middle_band, lower_band = talib.BBANDS(stock_data['Close'], timeperiod=90, nbdevup=2, nbdevdn=2)
    stock_data['BB_Upper'] = upper_band
    stock_data['BB_Lower'] = lower_band
    stock_data['Returns'] = stock_data['Close'].pct_change()
    stock_data['Volatility_30'] = stock_data['Returns'].rolling(90).std()
    stock_data.dropna(subset=['BB_Upper', 'BB_Lower', 'Volatility_30', 'Close_future'], inplace=True)

    if len(stock_data) < window_size:
        return None, None, None

    X_feat = stock_data[['BB_Upper', 'BB_Lower', 'Volatility_30']].values
    y_raw = stock_data['Close_future'].values
    dates = stock_data['Date'].values
    return X_feat, y_raw, dates

def load_full_data(ticker, start_date, end_date, window_size, prediction_horizon):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size, prediction_horizon)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)
    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)

def create_lstm_model(n_units, learning_rate, n_steps, n_features, n_layers=1):
    inputs = Input(shape=(n_steps, n_features))
    x = LSTM(n_units, activation='relu', return_sequences=(n_layers > 1))(inputs)
    if n_layers > 1:
        x = LSTM(n_units, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    return model

def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None,
                    initial_epochs=100, update_epochs=100,
                    initial_lr=0.0005, update_lr=0.0002,
                    n_units=16, batch_size=4):
    total_samples = len(X_samples)
    global train_ratio
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if X_train.size == 0 or X_val.size == 0:
        return None, np.nan

    n_steps_local = X_train.shape[1]
    n_features = X_train.shape[2]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    model = create_lstm_model(n_units, lr, n_steps_local, n_features, n_layers=1)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape

def rolling_training_early_stopping(ticker, start_date, end_date,
                                    n_steps=5, window_size=200, step_size=10,
                                    initial_epochs=100, update_epochs=100,
                                    initial_lr=0.0005, update_lr=0.0002,
                                    n_units=16, batch_size=4,
                                    prediction_horizon=126):
    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size, prediction_horizon)
    if X_full is None:
        return None, None

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        return None, None
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                                      initial_training=True, prev_weights=None,
                                      initial_epochs=initial_epochs, update_epochs=update_epochs,
                                      initial_lr=initial_lr, update_lr=update_lr,
                                      n_units=n_units, batch_size=batch_size)

    if model is None:
        return None, None

    weights_path = f"rolling_error_analysis_results/{ticker}_initial_6months.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                                          initial_training=False, prev_weights=weights_path,
                                          initial_epochs=initial_epochs, update_epochs=update_epochs,
                                          initial_lr=initial_lr, update_lr=update_lr,
                                          n_units=n_units, batch_size=batch_size)

        if model is None:
            break

        weights_path = f"rolling_error_analysis_results/{ticker}_iteration_{iteration}_6months.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])
        iteration += 1

    if len(predictions) == 0:
        return None, None

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0
    overall_mape = df_results['MAPE'].mean()

    return overall_mape, df_results

if __name__ == "__main__":
    mape, results_df = rolling_training_early_stopping(
        ticker,
        start_date=start_date,
        end_date=end_date,
        n_steps=n_steps,
        window_size=window_size,
        step_size=step_size,
        initial_epochs=initial_epochs,
        update_epochs=update_epochs,
        initial_lr=initial_lr,
        update_lr=update_lr,
        n_units=n_units,
        batch_size=batch_size,
        prediction_horizon=prediction_horizon
    )

    if mape is not None:
        print(f"[INFO] 6-month horizon rolling training for {ticker} completed. Overall MAPE: {mape:.2f}%")
        results_df.to_csv("rolling_error_analysis_results/cost_6months_results1.csv", index=False)
        print("[INFO] Detailed results saved to cost_6months_results1.csv")
    else:
        print("[WARN] Not enough data or no predictions made for COST with 6-month horizon.")
